In [ ]:
# imports here
#Treasure code is all below

import numpy as np
import random
from copy import deepcopy

TNumber: T00200799

Name: Ryan Madigan

Adjust the genetic algorithm to select the optimal treasure to take home...

You must maximise the value of the items selected subject to the following conditions

*   You must choose between 10 and 50 items (inclusive both ends)
*   Max Total Length   <= 240
*   Max Total Weight <= 300

Requires an answer for the seed 1969.

And an answer with a seed of their t number.

Results can be found at the bottom along with their parameters. The first block of results are for the 1969 seed. The second block are the resuts for the tnumber seed (200799)

In [ ]:
class treasure_item:
  value = 0
  length = 0
  weight = 0
  def __init__(self, max_value,max_length, max_weight,min_value = 0, min_length = 1, min_weight = 1):
    self.value = np.random.randint(min_value,max_value)
    self.length = np.random.randint(min_length,max_length)
    self.weight = np.random.randint(min_weight,max_weight)

  def display(self):
    print("Item Value "+ str(self.value) + " length " + str(self.length) + " weight " + str(self.weight))

In [ ]:
#This class generates treasure of random value, weight and length 
class treasure_hoard:
  item_count = 0
  items = []
  def __init__(self,total_item_count, seed):
    np.random.seed = seed
    self.max_value = np.random.randint(50,100)
    self.min_value = np.random.randint(0,3)
    self.max_length = np.random.randint(3,10)
    self.min_length = np.random.randint(1,3)
    self.max_weight = np.random.randint(3,20)
    self.min_weight = np.random.randint(1,3)

    for _ in range(total_item_count):
      self.items.append(treasure_item(self.max_value,self.max_length,self.max_weight,min_value=self.min_value, min_length= self.min_length, min_weight=self.min_weight))

In [ ]:
def rand_bit():
  if np.random.randint(2):
    return 0
  else:
    return 1

In [ ]:
pip install bitstring

In [ ]:
#Imports bitstring packages 
from bitstring import BitStream, BitArray

In [ ]:
#Creates a bitarray and fills it with 1000 zeroes 
b = BitArray(bin = '0'*1000)

In [ ]:
for _ in range(np.random.randint(0, 51)):
  b[np.random.randint(1000)] = True

All genetic algorithm code is below this cell

In [ ]:
def value_in_knapsack(individual, treasure):
  total = 0
  for i in range(len(individual)):
    if individual[i]:
      total += treasure.items[i].value
  return total

In [ ]:
def length_in_knapsack(individual, treasure):
  totalLength = 0
  for i in range(len(individual)):
    if individual[i]:
      totalLength += treasure.items[i].length
  return totalLength

In [ ]:
def weight_in_knapsack(individual, treasure):
  totalWeight = 0
  for i in range(len(individual)):
    if individual[i]:
      totalWeight += treasure.items[i].weight
  return totalWeight

In [ ]:
def total_items_in_knapsack(treasureTaken):
  total_number_of_items = 0
  for i in range(len(treasureTaken)):
    if treasureTaken[i]:
      total_number_of_items += 1
  return total_number_of_items

In [ ]:
#This method find the 'fitness' of the items that were taken 
def fitness_function(individual, treasure):
  max_length = 240
  max_weight = 300
  
  total_value = value_in_knapsack(individual, treasure)
  total_length = length_in_knapsack(individual, treasure)
  total_weight = weight_in_knapsack(individual, treasure)
  total_number_of_items = total_items_in_knapsack(individual)

  lengthOverMax = total_length - max_length
  weightOverMax = total_weight - max_weight
  numItemsOverMax = total_number_of_items - 50
  numItemsUnderMin = 10 - total_number_of_items 

  if (total_length <= max_length) and (total_weight <= max_weight) and (10 <= total_number_of_items <= 50) :
    return total_value
  else: 
    invalid_items = 0
    length = 0
    weight = 0

    if lengthOverMax > 0:
      length = lengthOverMax

    if weightOverMax > 0:
      weight = weightOverMax

    if numItemsOverMax > 0:
      invalid_items = numItemsOverMax

    elif numItemsUnderMin > 0:
      invalid_items = numItemsUnderMin

    fitness = (length + weight + invalid_items) * -1
    return fitness
  

Initially, if an item was invalid, it would return a 0. This would then cause the algorithm to return nothing but 0's. To counter this, I modified the fitness function to deal with invalid individuals by adding up the value, length, weight and returning it. When the algorithm is run, initially the values will start off invalid and will eventually become valid as the algorithm continues. 

In [ ]:
#Calculates the value of the treasure left behind
def value_left_behind(individual, treasure):
  return 5000 - fitness_function(individual, treasure)


In [ ]:
# Problem to be defined here as a class (or struct)
class problem:

  # Constructor
  def __init__(self, treasure):
    # Cost Function
    self.cost_function = value_left_behind

    # number of genes in chromosone
    self.number_of_genes = len(treasure.items)
    self.min_items = 10
    self.max_items = 50
    self.max_length = 240
    self.max_weight = 300

The problem class has been modified to fit with the knapsack problem in this assignment. 

In [ ]:
class parameters():
  def __init__(self):

    self.number_in_population = 200
    self.number_of_iterations = 300
    # Child rate is the proportion of new children relative to the population
    self.child_rate = 1
    self.number_of_children = self.number_in_population * self.child_rate

    #Crossover exploration Rate
    self.explore_rate = 0.2

    #Mutation parameters
    self.mutation_rate = 0.1

The mutation range and explore exploit have been removed as they are not needed for this problem 

In [ ]:
# individual
class individual():
  # Genotype or Chromosone
  chromosone = None

  # Cost
  cost = 0

  #Constructor

  def __init__(self, problem = None, treasure = None):
    self.chromosone = []
    if problem is not None:
      for i in range(problem.number_of_genes):
        self.chromosone.append(rand_bit())
      self.cost = problem.cost_function(self.chromosone, treasure)

  def crossover(self,other_parent,explore_rate):
      child1 = deepcopy(self)
      child2 = deepcopy(other_parent)

      cut_off_point = np.random.randint(0, len(self.chromosone))
      child1.chromosone = self.chromosone[:cut_off_point] + other_parent.chromosone[cut_off_point:]
      child2.chromosone = other_parent.chromosone[:cut_off_point] + self.chromosone[cut_off_point:]

      return child1, child2

  def mutate(self, mutation_rate):
    for i in range(len(self.chromosone)):
      if np.random.rand()<mutation_rate:
        if self.chromosone[i]:
          self.chromosone[i] = not self.chromosone[i]

The individual class is almost the same as before but the crossover method has been modified since this prolbem is not working with numbers like before. 

In [ ]:
#This method selects two parents through the use of the tournament selection 
def choose_distinct_pair_from_tournament(number_in_pop):
  index1 = tournamentSelection(number_in_pop, 10, population)
  index2 = tournamentSelection(number_in_pop, 10, population)
  if index1 == index2:
    return tournamentSelection(number_in_pop, 10)
  else:
    return index1,index2



In [ ]:
#This method chooses two random parents from the population
def choose_distinct_pair_from_random(number_in_pop, population):
  index1 = np.random.randint(number_in_pop)
  index2 = np.random.randint(number_in_pop)
  if index1 == index2:
    return choose_distinct_pair_from_random(number_in_pop)
  else:
    return index1, index2

In [ ]:
#This method carries out a tournament. It selects a random number of individuals from the population and then selects 
#the best individual through tournament selection
def tournamentSelection(numInPopulation, k, population):
  best = None
  for i in range(k):
    num = np.random.randint(1, numInPopulation)
    ind = population[np.random.randint(1, numInPopulation)]
    if best == None or fitness_function(ind, treasure) > fitness_function(best, treasure):
      best = ind
  return best[0]

This was my attempt at coding the tournament selection for choosing parents. The code is based off some pseudo code I found at : https://cstheory.stackexchange.com/questions/14758/tournament-selection-in-genetic-algorithms
Using this over random parent selection would most likely yield better results but I could not get it to work properly and could not use it. 

In [ ]:
def run_genetic( problem, parameters, treasure):
  # read problem
  cost_function = problem.cost_function

  #read parameters
  number_in_population = parameters.number_in_population
  number_of_iterations = parameters.number_of_iterations
  number_of_children = parameters.number_of_children
  crossover_explore_rate = parameters.explore_rate
  mutation_rate  = parameters.mutation_rate
  #placeholder for best individual
  
  best_solution = deepcopy(individual())
  best_solution.cost = np.infty

  population = []
  for i in range(number_in_population):
    new_individual = individual(problem, treasure)
    population.append(new_individual)
    if new_individual.cost < best_solution.cost:
      best_solution = deepcopy(new_individual)

  print(population[0].cost)


# Main Loop
  for iteration in range(number_of_iterations):
    children = []
    # Generate children
    while len(children) < number_of_children:
    
        # select parents
        parent1_index, parent2_index = choose_distinct_pair_from_random(number_in_population)
        parent1 = population[parent1_index]
        parent2 = population[parent2_index]

        # crossover
        child1, child2 = parent1.crossover(parent2,crossover_explore_rate)
        # mutate
        child1.mutate(mutation_rate)
        child2.mutate(mutation_rate)
        child1.cost = cost_function(child1.chromosone, treasure)
        child2.cost = cost_function(child2.chromosone, treasure)

        children.append(child1)
        children.append(child2)
        

    # Add children to population
    population += children

    # sort/reduce population to correct level
    population = sorted(population, key = lambda x:x.cost)

    if population[0].cost < best_solution.cost:
      best_solution = deepcopy(population[0])

    print("Value Left Behind " + str(best_solution.cost))

  return best_solution 
    

The run_genetic() method is very much the same except for removing a few unused parameters and making slight changes to the method calls as the treasure had to be passed in. 

The method returns the best value, the best value is determined by leaving as little value behind as possible. As the method runs, the value left behind decreases gradually. 

In [ ]:
#Treasure is the name given to the treasure generated from the 1969 seed. 
treasure = treasure_hoard(1000,200799)
prob = problem(treasure)
para = parameters()

In [ ]:
p = run_genetic(prob,para, treasure)

53633
Value Left Behind 46984
Value Left Behind 42590
Value Left Behind 38608
Value Left Behind 35619
Value Left Behind 32150
Value Left Behind 29113
Value Left Behind 26433
Value Left Behind 24524
Value Left Behind 22286
Value Left Behind 20582
Value Left Behind 19024
Value Left Behind 17381
Value Left Behind 16098
Value Left Behind 15037
Value Left Behind 13904
Value Left Behind 12944
Value Left Behind 11863
Value Left Behind 11088
Value Left Behind 10392
Value Left Behind 9586
Value Left Behind 9224
Value Left Behind 8725
Value Left Behind 8245
Value Left Behind 7867
Value Left Behind 7533
Value Left Behind 7162
Value Left Behind 6894
Value Left Behind 6557
Value Left Behind 6291
Value Left Behind 6055
Value Left Behind 5805
Value Left Behind 5684
Value Left Behind 5553
Value Left Behind 5438
Value Left Behind 5328
Value Left Behind 5208
Value Left Behind 5085
Value Left Behind 5017
Value Left Behind 5008
Value Left Behind 5002
Value Left Behind 2839
Value Left Behind 2748
Value Lef

--------------------------------------------------------------------------------
1969 Seed Results

1st Successful run of algorithm 
For the first successful run of the algorithm, the value left behind went from **11286** to **3401**. This was with the following paramters. 

*   Population = 100
*   Iterations = 100
*   Treasure Hoard = 1000
*   Parent Selection = Random Selection
*   Mutation Rate = 0.1

2nd Successful run of algorithm 
For the first successful run of the algorithm, the value left behind went from **9881** to **2774**. This was with the following paramters. 
This time the results improved without making any changes to the parameters below.  

*   Population = 100
*   Iterations = 100
*   Treasure Hoard = 1000
*   Parent Selection = Random Selection
*   Mutation Rate = 0.1

3rd Successful run of algorithm 
For the first successful run of the algorithm, the value left behind went from **16729** to **2655**. This was with the following paramters. 
This time the population was increased to 500 and there was a slight improvement over the previous run.   

*   Population = 500
*   Iterations = 100
*   Treasure Hoard = 1000
*   Parent Selection = Random Selection
*   Mutation Rate = 0.1

4th Successful run of algorithm 
For the first successful run of the algorithm, the value left behind went from **20502** to **1887**. This was with the following paramters. 
The number of iterations was increased to 200 and by simply doing this the there was a large improvement in comparison to the 3rd run of the algorithm. 

*   Population = 500
*   Iterations = 200
*   Treasure Hoard = 1000
*   Parent Selection = Random Selection
*   Mutation Rate = 0.1

5th Successful run of algorithm 
For the first successful run of the algorithm, the value left behind went from **25517** to **2481**. This was with the following paramters. 
This time the mutation rate was changed to 0.2 but yielded no improvement over the previous result. 

*   Population = 500
*   Iterations = 200
*   Treasure Hoard = 1000
*   Parent Selection = Random Selection
*   Mutation Rate = 0.2

6th Successful run of algorithm 
For the first successful run of the algorithm, the value left behind went from **29225** to **2108**. This was with the following paramters. 
I thought that by increasing the iterations it would lead to better results but there was no improvement. The best solution improved very slowly after a certain point.

*   Population = 500
*   Iterations = 500
*   Treasure Hoard = 1000
*   Parent Selection = Random Selection
*   Mutation Rate = 0.2

In order to get better results, then it would be best to implement a different method of parent selection. These runs only use random selection. Results would most likely improve if using the roulette wheel or tournament selection. I made an attempt at implementing tournament selection but could not get it to work and had to revert back to random parent selection

7th Successful run of algorithm 
For the first successful run of the algorithm, the value left behind went from **34420** to **1961**. This was with the following paramters. 
For this I decided to change the population back to 100, the mutation rate back to 0.1. This yielded better results but not as good as a previous run. It seems that a lower mutation rate works better and that the number of iterations has a bit of influence on the results. 

*   Population = 100
*   Iterations = 200
*   Treasure Hoard = 1000
*   Parent Selection = Random Selection
*   Mutation Rate = 0.1

8th Successful run of algorithm 
For the first successful run of the algorithm, the value left behind went from **39580** to **1940**. This was with the following paramters. 
This time I changed the iterations to 150 and got a slight improvement over the 7th run but not enough to beat the 4th run
*   Population = 100
*   Iterations = 150
*   Treasure Hoard = 1000
*   Parent Selection = Random Selection
*   Mutation Rate = 0.1


--------------------------------------------------------------------------------
TNumber (200799) Seed Results 

1st Successful run of algorithm using T Number seed. For the first successful run of the algorithm, the value left behind went from **37763** to **2196**. This was with the following paramters.


*   Population = 100
*   Iterations = 100
*   Treasure Hoard = 1000
*   Parent Selection = Random Selection
*   Mutation Rate = 0.1

2nd Successful run of algorithm using T Number seed. For the first successful run of the algorithm, the value left behind went from **42823** to **2348**. This was with the following paramters. I made no changes to parameters. Unfortuantely there was no improvement over the previous run. Next run onwards I will begin making changes to parameters based on what I learned from the results from the 1969 seed. 

*   Population = 100
*   Iterations = 100
*   Treasure Hoard = 1000
*   Parent Selection = Random Selection
*   Mutation Rate = 0.1

3rd Successful run of algorithm using T Number seed. For the first successful run of the algorithm, the value left behind went from **46319** to **1620**. This was with the following paramters. This time I increased both the population and iterations to 200 and this yielded my best results yet. With a value left behind of **1620**. 

*   Population = 200
*   Iterations = 200
*   Treasure Hoard = 1000
*   Parent Selection = Random Selection
*   Mutation Rate = 0.1

4th Successful run of algorithm using T Number seed. For the first successful run of the algorithm, the value left behind went from **53633** to **1582**. This was with the following paramters. For this run I increased the iterations to 300 and was able to get a slight improvement over the previous run. Increasing iteratios doesn't guarantee an improvement as the improvements start to slow down a lot while running the algorithm. To get a really good improvement it would be necessary to change the parent selection. If I were able to get the tournament selection to work then it would guarantee picking the best indiviudal out of the random selection. As it is now it is vey luck based since there is no control over parent selection. 

*   Population = 200
*   Iterations = 300
*   Treasure Hoard = 1000
*   Parent Selection = Random Selection
*   Mutation Rate = 0.1








